In [1]:
#basic packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression    
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.decomposition import PCA

import time

from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [9]:
df = pd.read_csv('data/Combined_News_DJIA_Prices_With_TPlusLabels.csv')

In [10]:
def create_dataset(x_in, y_in, window=10 ):
    x = []
    y = []

    #print("x",x_in.shape,"y : ", y_in.shape[0])
    for i in range(window, x_in.shape[0]):
           x.append(x_in[i-window:i].flatten())
           y.append(y_in[i])  

    x = np.array(x)    
    y = np.array(y)    
    
    #print(x.shape, y.shape)
    return x,y

In [11]:
def save_results_to_csv(results, colnames, file_name_suffix ):
    results_df = pd.DataFrame(results,columns=colnames)
    fname=file_name_suffix+str(time.time())+'.csv'
    results_df.to_csv(fname)

In [12]:
#creating samples from the testing/training dataset
def create_dataset_reg(x_in, price, label, window=10):
    x = []
    y = []
    z = []
    
    for i in range(window, x_in.shape[0]):        
        x.append(x_in[i-window:i].flatten())
        y.append(price[i])
        z.append(label[i])  
    x = np.array(x)
    y = np.array(y)
    z = np.array(z)
    return x,y,z

In [13]:
def direction(price_tplusone, price_t):
    if price_tplusone > price_t :
        return 1
    return 0

In [14]:
def accuracy_of_direction(y_predicted, label_test,price_test):                
                label_predicted = []            
                label_predicted.append(direction(y_predicted[0],price_test[0]))
                for index in range(1,len(y_predicted)):
                     label_predicted.append(direction(y_predicted[index], y_predicted[index-1]))
                    
                
                ts_acc_pca = accuracy_score(label_test,label_predicted)
                return ts_acc_pca

In [16]:
def run_predictions_classifier(dfCombined,y_arr,
                              attribsSet = [['Close'],['Close','Volume'],['Close','Volume','High','Low']],
                              train_percentages=[0.1,0.2,0.3,0.5,0.6,0.8],
                              window_sizes = [3,5,10,15,20,25,30,40,50,55,60,63,65,70,80,90,100,110,120,130,150], 
                              csvPrefix='results_classification'
                              ): 
    #attribsSet = [['Close'],['Close','Volume'],['Close','Volume','High','Low']]
    resultsList=['Attributes','Train Per','Window size','Classifer','Using PCA','Accuracy']
    #train_percentages=[0.1,0.2,0.3,0.5,0.6,0.8]
    #train_percentages=[0.2,0.5,0.7]
    #window_sizes = [3,5,10,15,20,25,30,40,50,55,60,63,65,70,80,90,100,110,120,130,150]
    #window_sizes = range(5,205,5)
    modelList = []
    modelList.append(("LogisticReg",LogisticRegression(random_state=42)))
    modelList.append(("RandomForestClassifier",RandomForestClassifier(random_state=42)))
    modelList.append(("SVC",SVC(random_state=42)))
    modelList.append(("DecisionTreeClassifier",DecisionTreeClassifier(random_state=42)))
    modelList.append(("KNeighborsClassifier",KNeighborsClassifier(10)))
    modelList.append(("GaussianNB",GaussianNB()))
    modelList.append(("XGBClassifier",XGBClassifier()))
    modelList.append(("AdaBoostClassifier",AdaBoostClassifier()))    
    ts_acc_best=-2.0
    best=[]    
    #attribsSet = [['Close','Volume','High','Low']]
    for attribIndex in range(0,len(attribsSet)):    
        closeVoldf = dfCombined[attribsSet[attribIndex]].values                                                
        for train_per in train_percentages:            
            for window in window_sizes:
                #print(dataset_train.shape)
                #print(dataset_test.shape)


                dataset_train = np.array(closeVoldf[:int(closeVoldf.shape[0]*train_per)])
                y_train = np.array(y_arr[:int(y_arr.shape[0]*train_per)])
                dataset_test = np.array(closeVoldf[int(closeVoldf.shape[0]*train_per)-window:])
                y_test = np.array(y_arr[int(y_arr.shape[0]*train_per)-window:])

                min_max_scaler = MinMaxScaler(feature_range=(0,1))
                dataset_train = min_max_scaler.fit_transform(dataset_train)
                dataset_test = min_max_scaler.transform(dataset_test)

                #print(dataset_train.shape,y_train.shape)
                #print(dataset_test.shape,y_test.shape)

                x_train, y_train = create_dataset(dataset_train,y_train,window)
                x_test, y_test   = create_dataset(dataset_test,y_test,window)


                for name, classifier in modelList:                                      
                    classifier.fit(x_train,y_train)
                    ytr_pred = classifier.predict(x_train)
                    yts_pred = classifier.predict(x_test)

                    tr_acc = accuracy_score(y_train,ytr_pred)
                    ts_acc= accuracy_score(y_test,yts_pred)
                   # data[name] = [attribsSet[attribIndex],tr_acc,ts_acc]
                    print('Completed ',train_per,window,name,attribsSet[attribIndex]," Score[test]",ts_acc)
                    resultsList.append([attribsSet[attribIndex],train_per,window,name,'N',ts_acc])
                    if(ts_acc_best <= ts_acc) :
                        ts_acc_best = ts_acc
                        best = [train_per,window,name,attribsSet[attribIndex],'No PCA']

                    pca_final = PCA(0.90)
                    df_train_pca = pca_final.fit_transform(x_train)
                    df_test_pca = pca_final.transform(x_test)

                    classifier.fit(df_train_pca,y_train)
                    ytr_pred_pca = classifier.predict(df_train_pca)
                    yts_pred_pca = classifier.predict(df_test_pca)
                    tr_acc_pca = accuracy_score(y_train,ytr_pred_pca)
                    ts_acc_pca = accuracy_score(y_test,yts_pred_pca)

                    resultsList.append([attribsSet[attribIndex],train_per,window,name,'Y',ts_acc_pca])
                    print('Completed with PCA with ',pca_final.n_components_,' components',train_per,window,name,attribsSet[attribIndex]," Score[test]",ts_acc_pca)
                    if(ts_acc_best <= ts_acc_pca) :
                        ts_acc_best = ts_acc_pca
                        best = [train_per,window,name,attribsSet[attribIndex],'With PCA']
    print('Best',ts_acc_best,best)          
    save_results_to_csv(resultsList[6:],['Attributes','Train Per','Window size','Classifer','Using PCA','Accuracy'],csvPrefix)

In [17]:
def run_predictions_regression(dfCombined,price_arr,z_arr,
                              attribsSet = [['Close'],['Close','Volume'],['Close','Volume','High','Low']],
                               train_percentages=[0.1,0.2,0.3,0.5,0.6,0.8],
                               window_sizes = [3,5,10,15,20,25,30,40,50,55,60,63,65,70,80,90,100,110,120,130,150],
                               csvPrefix='results_regression'                               
                              ):
    #attribsSet = [['Close'],['Close','Volume'],['Close','Volume','High','Low']]
    #attribsSet = [['Close','Volume','High','Low']]
    ts_acc_best=-2.0
    best=[]
    resultsReg=[]
    for attribIndex in range(0,len(attribsSet)):    
        closeVoldf = dfCombined[attribsSet[attribIndex]].values                                
        #train_percentages=[0.1,0.2,0.3,0.5,0.6,0.8]
        #train_percentages=[0.2,0.5,0.7]
        #window_sizes = [3,5,10,15,20,25,30,40,50,55,60,63,65,70,80,90,100,110,120,130,150]
        modelList = []
        modelList.append(("LinearRegression",LinearRegression()))

        ts_acc_best=-2.0
        best=[]
        results={}        

        for train_per in train_percentages:            
            for window in window_sizes:

                dataset_train = np.array(closeVoldf[:int(closeVoldf.shape[0]*train_per)])
                price_arr_train = np.array(price_arr[:int(price_arr.shape[0]*train_per)])
                z_arr_train = np.array(z_arr[:int(z_arr.shape[0]*train_per)])

                dataset_test = np.array(closeVoldf[int(closeVoldf.shape[0]*train_per)-window:])
                price_arr_test = np.array(price_arr[int(price_arr.shape[0]*train_per)-window:])
                z_arr_test = np.array(z_arr[int(z_arr.shape[0]*train_per)-window:])

                price_arr_train = price_arr_train.reshape(-1,1)
                price_arr_test = price_arr_test.reshape(-1,1)

                min_max_scaler = MinMaxScaler(feature_range=(0,1))
                dataset_train = min_max_scaler.fit_transform(dataset_train)
                price_arr_train = MinMaxScaler(feature_range=(0,1)).fit_transform(price_arr_train)
                dataset_test = min_max_scaler.fit_transform(dataset_test)
                price_arr_test = MinMaxScaler(feature_range=(0,1)).fit_transform(price_arr_test)


                x_train, price_train, label_train = create_dataset_reg(dataset_train, price_arr_train, z_arr_train,window)
                x_test, price_test, label_test = create_dataset_reg(dataset_test, price_arr_test, z_arr_test,window)


                for name, regression in modelList: 

                    regression.fit(x_train,price_train)            
                    y_pred_test = regression.predict(x_test)                                
                    ts_acc = accuracy_of_direction(y_pred_test,label_test,price_test)                
                    resultsReg.append([attribsSet[attribIndex],train_per,window,name,'N',ts_acc])
                    if(ts_acc_best <= ts_acc) :
                            ts_acc_best = ts_acc
                            best = [train_per,window,attribsSet[attribIndex],'No PCA']


                    pca_final = PCA(0.95)
                    df_train_pca = pca_final.fit_transform(x_train)
                    df_test_pca = pca_final.transform(x_test)
                    regression.fit(df_train_pca,price_train)            
                    y_pred_test_pca = regression.predict(df_test_pca)           
                    print(y_pred_test_pca.shape,price_test.shape)
                    ts_acc_pca = accuracy_of_direction(y_pred_test_pca,label_test,price_test)
                    resultsReg.append([attribsSet[attribIndex],train_per,window,name,'Y',ts_acc_pca])

                    if(ts_acc_best <= ts_acc_pca) :
                            ts_acc_best = ts_acc_pca
                            best = [train_per,window,attribsSet[attribIndex],'No PCA']

                    print('Completed ',name,attribsSet[attribIndex],train_per,window,'Acc ',
                          ts_acc,'Acc PCA components ',pca_final.n_components_,ts_acc_pca)
    save_results_to_csv(resultsReg,
                    ['Attributes','Train Per','Window size','Classifer','Using PCA','Accuracy']
                    ,csvPrefix)
                
    print('Best ',best,'Acc',ts_acc_best)                      

In [18]:
def create_models(window,attribsSet,attribIndex,output_size=1):  
            result=[]
            ann_name='model_1_2hidden_relu_adam'
            #building the model
            model = Sequential()
            model.add(Dense(window, activation='relu', input_shape=(window*len(attribsSet[attribIndex]),)))
            model.add(Dense(int(window*1.5), activation='relu'))
            model.add(Dense(int(window*1.5), activation='relu'))                            
            model.add(Dense(output_size, activation='sigmoid'))            
            model.compile(optimizer='adam', loss='mean_squared_error')
            result.append([ann_name,model])
            
            ann_name='model_2_2hidden_tanh_adam'
            #building the model
            model = Sequential()
            model.add(Dense(window, activation='tanh', input_shape=(window*len(attribsSet[attribIndex]),)))
            model.add(Dense(int(window*1.5), activation='tanh'))
            model.add(Dense(int(window*1.5), activation='tanh'))                            
            model.add(Dense(output_size, activation='tanh'))            
            model.compile(optimizer='adam', loss='mean_squared_error')
            result.append([ann_name,model])
            
            ann_name='model_3_2hidden_softmax_adam'
            #building the model
            model = Sequential()
            model.add(Dense(window, activation='softmax', input_shape=(window*len(attribsSet[attribIndex]),)))
            model.add(Dense(int(window*1.5), activation='softmax'))
            model.add(Dense(int(window*1.5), activation='softmax'))                            
            model.add(Dense(output_size, activation='softmax'))            
            model.compile(optimizer='adam', loss='mean_squared_error')
            result.append([ann_name,model])
            
            ann_name='model_4_2hidden_softmax_sgd'
            #building the model
            model = Sequential()
            model.add(Dense(window, activation='softmax', input_shape=(window*len(attribsSet[attribIndex]),)))
            model.add(Dense(int(window*1.5), activation='softmax'))
            model.add(Dense(int(window*1.5), activation='softmax'))                            
            model.add(Dense(output_size, activation='softmax'))            
            model.compile(optimizer='sgd', loss='mean_squared_error')
            result.append([ann_name,model])
            
            ann_name='model_5_2hidden_softmax_adagrad'
            #building the model
            model = Sequential()
            model.add(Dense(window, activation='softmax', input_shape=(window*len(attribsSet[attribIndex]),)))
            model.add(Dense(int(window*1.5), activation='softmax'))
            model.add(Dense(int(window*1.5), activation='softmax'))                            
            model.add(Dense(output_size, activation='softmax'))                        
            model.compile(optimizer='adagrad', loss='mean_squared_error')
            result.append([ann_name,model])
            
             
            
            ann_name='model_6_4hidden_relu_adam'
            #building the model
            model = Sequential()
            model.add(Dense(window, activation='relu', input_shape=(window*len(attribsSet[attribIndex]),)))
            model.add(Dense(int(window*1.5), activation='relu'))
            model.add(Dense(int(window*2), activation='relu'))                            
            model.add(Dense(int(window*2), activation='relu'))  
            model.add(Dense(int(window*1.5), activation='relu'))
            model.add(Dense(output_size, activation='relu'))            
            model.compile(optimizer='adam', loss='mean_squared_error')
            result.append([ann_name,model])
            
            
            
            ann_name='model_7_4hidden_tanh_relu_adam'
            #building the model
            model = Sequential()
            model.add(Dense(window, activation='tanh', input_shape=(window*len(attribsSet[attribIndex]),)))
            model.add(Dense(int(window*1.5), activation='relu'))
            model.add(Dense(int(window*2), activation='relu'))                            
            model.add(Dense(int(window*2), activation='tanh'))  
            model.add(Dense(int(window*1.5), activation='tanh'))
            model.add(Dense(output_size, activation='relu'))            
            model.compile(optimizer='adam', loss='mean_squared_error')
            result.append([ann_name,model])
            
            
            ann_name='model_8_4hidden_tanh_softmax_adagrad'
            #building the model
            model = Sequential()
            model.add(Dense(window, activation='tanh', input_shape=(window*len(attribsSet[attribIndex]),)))
            model.add(Dense(int(window*1.5), activation='softmax'))
            model.add(Dense(int(window*2), activation='softmax'))                            
            model.add(Dense(int(window*2), activation='tanh'))  
            model.add(Dense(int(window*1.5), activation='tanh'))
            model.add(Dense(output_size, activation='softmax'))            
            model.compile(optimizer='adagrad', loss='mean_squared_error')
            result.append([ann_name,model])
            
            return result

In [19]:
def run_predictions_ann(dfCombined,price_arr,z_arr,
                        attribsSet=[['Close','Volume','High','Low']],
                        epochsList=[200,300],
                        train_percentages=[0.3,0.5,0.7],
                        window_sizes = [5,10,15,20,30,50,60,65,80,90,100,120,150],
                        csvPrefix='results_ann'):
    #attribsSet = [['Close','Volume','High','Low']]
    #attribsSet = [['Close','Volume','High','Low']]
    ts_acc_best=-2.0
    best=[]
    #epochsList=[200,300]
    output_size=1
    for attribIndex in range(0,len(attribsSet)):    
        closeVoldf = dfCombined[attribsSet[attribIndex]].values  
        #train_percentages=[0.3,0.5,0.7]    
        #window_sizes = [5,10,15,20,30,50,60,65,80,90,100,120,150]        
        for train_per in train_percentages:  
            resultsANN=[]            
            for window in window_sizes:
                dataset_train = np.array(closeVoldf[:int(closeVoldf.shape[0]*train_per)])
                dataset_test = np.array(closeVoldf[int(closeVoldf.shape[0]*train_per)-window:])
                price_arr_train = np.array(price_arr[:int(price_arr.shape[0]*train_per)])
                z_arr_train = np.array(z_arr[:int(z_arr.shape[0]*train_per)])        
                price_arr_test = np.array(price_arr[int(price_arr.shape[0]*train_per)-window:])
                z_arr_test = np.array(z_arr[int(z_arr.shape[0]*train_per)-window:])

                price_arr_train = price_arr_train.reshape(-1,1)
                price_arr_test = price_arr_test.reshape(-1,1)
                min_max_scaler = MinMaxScaler(feature_range=(0,1))
                dataset_train = min_max_scaler.fit_transform(dataset_train)
                price_arr_train = min_max_scaler.fit_transform(price_arr_train)
                dataset_test = min_max_scaler.fit_transform(dataset_test)
                price_arr_test = min_max_scaler.fit_transform(price_arr_test)


                x_train, y_train, z_train = create_dataset_reg(dataset_train,price_arr_train,z_arr_train, window)
                x_test, y_test, z_test = create_dataset_reg(dataset_test,price_arr_test,z_arr_test, window)
                start = time.time()
                print('Creating models for window size ',window)
                modelList = create_models(window,attribsSet,attribIndex)
                print('Created models for window size ',window,' in ',(time.time()-start))
                for name, model in modelList:
                    for epoch in epochsList:
                        print('Running model ',name,' with epoch ',epoch,attribsSet[attribIndex],window,train_per)
                        start = time.time()
                        #training the model
                        #history = model.fit(x_train, y_train, epochs=50, batch_size = 2)
                        history = model.fit(x_train, y_train, validation_split=0.15, epochs=epoch, batch_size=40, verbose=0,shuffle=False)
                        #print(model.summary())
                        y_test_pred=model.predict(x_test)
                        ts_acc = accuracy_of_direction(y_test_pred,z_test,y_test)
                        print('Completed model ',name,' with epoch ',epoch,
                              [attribsSet[attribIndex],train_per,window,name,epoch,ts_acc],
                              'Time ',(time.time()-start)
                             )

                        resultsANN.append([attribsSet[attribIndex],train_per,window,name,epoch,ts_acc])
                        if(ts_acc_best <= ts_acc) :
                                ts_acc_best = ts_acc
                                best = [train_per,window,attribsSet[attribIndex],name]    
            save_results_to_csv(resultsANN,
                        ['Attributes','Train Per','Window size','ModelName','Epoch','Accuracy']
                        ,csvPrefix+str(train_per))                      

    print('Best ',best,'Acc',ts_acc_best)  
    

In [ ]:
dfTplus5=df.dropna(subset=['LabelTPlus5'])
prices_tplus5= np.array(dfTplus5['CloseTPlus5'])
y_arr_tplus5 = np.array(dfTplus5['LabelTPlus5'])
print('Performing T plus 5')
run_predictions_classifier(dfTplus5,y_arr_tplus5,                           
                          csvPrefix='results_classification_tplus5_'
                           )
run_predictions_regression(dfTplus5,prices_tplus5,y_arr_tplus5,                           
                           csvPrefix='results_regression_tplus5_'
                           )
run_predictions_ann(dfTplus5,prices_tplus5,y_arr_tplus5,                           
                           csvPrefix='results_ann_tplus5_'
                           )

Performing T plus 5
Completed  0.1 3 LogisticReg ['Close']  Score[test] 0.4568868980963046
Completed with PCA with  1  components 0.1 3 LogisticReg ['Close']  Score[test] 0.4568868980963046
Completed  0.1 3 RandomForestClassifier ['Close']  Score[test] 0.5151175811870101
Completed with PCA with  1  components 0.1 3 RandomForestClassifier ['Close']  Score[test] 0.5257558790593505
Completed  0.1 3 SVC ['Close']  Score[test] 0.5106382978723404
Completed with PCA with  1  components 0.1 3 SVC ['Close']  Score[test] 0.5111982082866742
Completed  0.1 3 DecisionTreeClassifier ['Close']  Score[test] 0.5151175811870101
Completed with PCA with  1  components 0.1 3 DecisionTreeClassifier ['Close']  Score[test] 0.5257558790593505
Completed  0.1 3 KNeighborsClassifier ['Close']  Score[test] 0.5089585666293394
Completed with PCA with  1  components 0.1 3 KNeighborsClassifier ['Close']  Score[test] 0.5078387458006719
Completed  0.1 3 GaussianNB ['Close']  Score[test] 0.5162374020156775
Completed with

Completed with PCA with  1  components 0.1 25 XGBClassifier ['Close']  Score[test] 0.46360582306830905
Completed  0.1 25 AdaBoostClassifier ['Close']  Score[test] 0.45464725643896975
Completed with PCA with  1  components 0.1 25 AdaBoostClassifier ['Close']  Score[test] 0.458006718924972
Completed  0.1 30 LogisticReg ['Close']  Score[test] 0.4568868980963046
Completed with PCA with  2  components 0.1 30 LogisticReg ['Close']  Score[test] 0.4568868980963046
Completed  0.1 30 RandomForestClassifier ['Close']  Score[test] 0.4563269876819709
Completed with PCA with  2  components 0.1 30 RandomForestClassifier ['Close']  Score[test] 0.4652855543113102
Completed  0.1 30 SVC ['Close']  Score[test] 0.4568868980963046
Completed with PCA with  2  components 0.1 30 SVC ['Close']  Score[test] 0.4568868980963046
Completed  0.1 30 DecisionTreeClassifier ['Close']  Score[test] 0.5027995520716685
Completed with PCA with  2  components 0.1 30 DecisionTreeClassifier ['Close']  Score[test] 0.464725643896

Completed  0.1 63 XGBClassifier ['Close']  Score[test] 0.5369540873460247
Completed with PCA with  4  components 0.1 63 XGBClassifier ['Close']  Score[test] 0.49048152295632697
Completed  0.1 63 AdaBoostClassifier ['Close']  Score[test] 0.5027995520716685
Completed with PCA with  4  components 0.1 63 AdaBoostClassifier ['Close']  Score[test] 0.5403135498320268
Completed  0.1 65 LogisticReg ['Close']  Score[test] 0.4568868980963046
Completed with PCA with  4  components 0.1 65 LogisticReg ['Close']  Score[test] 0.4568868980963046
Completed  0.1 65 RandomForestClassifier ['Close']  Score[test] 0.5375139977603584
Completed with PCA with  4  components 0.1 65 RandomForestClassifier ['Close']  Score[test] 0.5408734602463606
Completed  0.1 65 SVC ['Close']  Score[test] 0.5274356103023516
Completed with PCA with  4  components 0.1 65 SVC ['Close']  Score[test] 0.46976483762597987
Completed  0.1 65 DecisionTreeClassifier ['Close']  Score[test] 0.5089585666293394
Completed with PCA with  4  com

Completed  0.1 110 XGBClassifier ['Close']  Score[test] 0.5061590145576708
Completed with PCA with  6  components 0.1 110 XGBClassifier ['Close']  Score[test] 0.5005599104143337
Completed  0.1 110 AdaBoostClassifier ['Close']  Score[test] 0.4669652855543113
Completed with PCA with  6  components 0.1 110 AdaBoostClassifier ['Close']  Score[test] 0.480963045912654
Completed  0.1 120 LogisticReg ['Close']  Score[test] 0.46360582306830905
Completed with PCA with  7  components 0.1 120 LogisticReg ['Close']  Score[test] 0.46248600223964165
Completed  0.1 120 RandomForestClassifier ['Close']  Score[test] 0.5375139977603584
Completed with PCA with  7  components 0.1 120 RandomForestClassifier ['Close']  Score[test] 0.5151175811870101
Completed  0.1 120 SVC ['Close']  Score[test] 0.46360582306830905
Completed with PCA with  7  components 0.1 120 SVC ['Close']  Score[test] 0.46472564389697646
Completed  0.1 120 DecisionTreeClassifier ['Close']  Score[test] 0.49384098544232924
Completed with PCA

Completed  0.2 10 GaussianNB ['Close']  Score[test] 0.46851385390428213
Completed with PCA with  1  components 0.2 10 GaussianNB ['Close']  Score[test] 0.48614609571788414
Completed  0.2 10 XGBClassifier ['Close']  Score[test] 0.4722921914357683
Completed with PCA with  1  components 0.2 10 XGBClassifier ['Close']  Score[test] 0.5232997481108312
Completed  0.2 10 AdaBoostClassifier ['Close']  Score[test] 0.4811083123425693
Completed with PCA with  1  components 0.2 10 AdaBoostClassifier ['Close']  Score[test] 0.473551637279597
Completed  0.2 15 LogisticReg ['Close']  Score[test] 0.5371536523929471
Completed with PCA with  1  components 0.2 15 LogisticReg ['Close']  Score[test] 0.5371536523929471
Completed  0.2 15 RandomForestClassifier ['Close']  Score[test] 0.52455919395466
Completed with PCA with  1  components 0.2 15 RandomForestClassifier ['Close']  Score[test] 0.5283375314861462
Completed  0.2 15 SVC ['Close']  Score[test] 0.47544080604534006
Completed with PCA with  1  components

Completed with PCA with  2  components 0.2 50 DecisionTreeClassifier ['Close']  Score[test] 0.5044080604534005
Completed  0.2 50 KNeighborsClassifier ['Close']  Score[test] 0.46914357682619645
Completed with PCA with  2  components 0.2 50 KNeighborsClassifier ['Close']  Score[test] 0.4722921914357683
Completed  0.2 50 GaussianNB ['Close']  Score[test] 0.5371536523929471
Completed with PCA with  2  components 0.2 50 GaussianNB ['Close']  Score[test] 0.5371536523929471
Completed  0.2 50 XGBClassifier ['Close']  Score[test] 0.5258186397984886
Completed with PCA with  2  components 0.2 50 XGBClassifier ['Close']  Score[test] 0.4968513853904282
Completed  0.2 50 AdaBoostClassifier ['Close']  Score[test] 0.46662468513853905
Completed with PCA with  2  components 0.2 50 AdaBoostClassifier ['Close']  Score[test] 0.47103274559193953
Completed  0.2 55 LogisticReg ['Close']  Score[test] 0.5390428211586902
Completed with PCA with  2  components 0.2 55 LogisticReg ['Close']  Score[test] 0.537153652

Completed  0.2 80 RandomForestClassifier ['Close']  Score[test] 0.5232997481108312
Completed with PCA with  2  components 0.2 80 RandomForestClassifier ['Close']  Score[test] 0.5377833753148614
Completed  0.2 80 SVC ['Close']  Score[test] 0.5371536523929471
Completed with PCA with  2  components 0.2 80 SVC ['Close']  Score[test] 0.5371536523929471
Completed  0.2 80 DecisionTreeClassifier ['Close']  Score[test] 0.5377833753148614
Completed with PCA with  2  components 0.2 80 DecisionTreeClassifier ['Close']  Score[test] 0.48740554156171284
Completed  0.2 80 KNeighborsClassifier ['Close']  Score[test] 0.5384130982367759
Completed with PCA with  2  components 0.2 80 KNeighborsClassifier ['Close']  Score[test] 0.5371536523929471
Completed  0.2 80 GaussianNB ['Close']  Score[test] 0.5264483627204031
Completed with PCA with  2  components 0.2 80 GaussianNB ['Close']  Score[test] 0.5371536523929471
Completed  0.2 80 XGBClassifier ['Close']  Score[test] 0.5283375314861462
Completed with PCA wi

In [ ]:
dfTplus20=df.dropna(subset=['LabelTPlus20'])
prices_tplus20= np.array(dfTplus20['CloseTPlus20'])
y_arr_tplus20 = np.array(dfTplus20['LabelTPlus20'])
print('Performing T plus 20')
run_predictions_classifier(dfTplus20,y_arr_tplus20,                           
                          csvPrefix='results_classification_tplus20_'
                           )
run_predictions_regression(dfTplus20,prices_tplus20,y_arr_tplus20,                           
                           csvPrefix='results_regression_tplus20_'
                           )
run_predictions_ann(dfTplus20,prices_tplus20,y_arr_tplus20,                           
                           csvPrefix='results_ann_tplus20_'
                           )

In [ ]:
dfTplus60=df.dropna(subset=['LabelTPlus60'])
prices_tplus60= np.array(dfTplus60['CloseTPlus60'])
y_arr_tplus60 = np.array(dfTplus60['LabelTPlus60'])
print('Performing T plus 60')
run_predictions_classifier(dfTplus60,y_arr_tplus60,                           
                          csvPrefix='results_classification_tplus60_'
                           )
run_predictions_regression(dfTplus60,prices_tplus60,y_arr_tplus60,                           
                           csvPrefix='results_regression_tplus60_'
                           )
run_predictions_ann(dfTplus60,prices_tplus60,y_arr_tplus60,                           
                           csvPrefix='results_ann_tplus60_'
                           )